### Install packages

In [ ]:
!pip install -U "qai_hub_models[llama-v3_2-3b-chat-quantized]" -q

In [ ]:
!pip install transformers huggingface_hub -q


In [ ]:
!pip install torch==2.4.1 torchvision torchaudio -q

In [ ]:
!pip install -U "huggingface_hub[cli]" -q

### AI Hub

In [ ]:
# https://github.com/quic/ai-hub-apps/tree/main/tutorials/llm_on_genie
# https://github.com/quic/ai-hub-apps/tree/main/apps/android/ChatApp

In [ ]:
# set tokens
!qai-hub configure --api_token <your_token>

In [ ]:
# Download llama - write authority is needed to download model;;
!huggingface-cli download meta-llama/Llama-3.2-3B-Instruct

In [ ]:
# device : 
#       For Android - "Snapdragon 8 Gen 3 QRD" "Snapdragon 8 Elite QRD"
#       For Windows - "Snapdragon 8 Gen 3 CRD" "Snapdragon 8 Elite CRD"

# --context-length 2048 for 3B model
!python -m qai_hub_models.models.llama_v3_2_3b_chat_quantized.export --device "Snapdragon 8 Elite QRD" --skip-inferencing --skip-profiling --output-dir llama3.2_bundle --context-length 2048

In [ ]:
# Remove intermediate assets
!rm -rf genie_bundle/{prompt,token}

### Profile each model

In [ ]:
import qai_hub as hub

name = 'custom4_dw_unet'
torch_model = DWCustomUNet2(in_channels=1, out_channels=1)
torch_model.eval()

# Step 1: Trace model
input_shape = (1, 1, 512, 512)
example_input = torch.rand(input_shape)
traced_torch_model = torch.jit.trace(torch_model, example_input)

# Step 2: Compile model
compile_job = hub.submit_compile_job( # AP에 최적화된 형태로 compile
    model=traced_torch_model,
    device=hub.Device("Snapdragon 8 Elite QRD"),
    input_specs=dict(image=input_shape),
)

# Step 3: Profile on cloud-hosted device
target_model = compile_job.get_target_model()
profile_job = hub.submit_profile_job( # compile된 모델을 가져와서 실제 HW에 올려서 prifiling
    model=target_model,
    device=hub.Device("Snapdragon 8 Elite QRD"),
)

random_input = np.random.rand(1, 1, 512, 512).astype(np.float32)

# Run inference using the on-device model on the input image
inference_job = hub.submit_inference_job(
    model=target_model,
    device=hub.Device("Snapdragon 8 Elite QRD"),
    inputs=dict(image=[random_input]),
)

# # Step 6: Download model
# target_model = compile_job.get_target_model()
# target_model.download(f"{name}.tflite")
